# Cinetel Data Cleaning

Cinetel data scraped daily from their boxoffice published everyday at this [link](https://www.cinetel.it/pages/boxoffice.php?edperiodo=aWVyaQ==).

## Reading In Files

In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv(r"E:\data_analysis_python\cinetel\cinetel_.csv")
df1.head()

,Pos.,Titolo,Prima Progr.,Nazione,Distribuzione,Incasso,Presenze,Incasso al 16/11/2023,Presenze al 16/11/2023,2023-11-17 15:31:50.351708
0,1,C'E' ANCORA DOMANI,'2023-10-26'26/10/2023,ITA,VISION DISTRIBUTION,449991.06,66700,15231636.85,2202169,2023-11-17 15:31:50.351708
1,2,HUNGER GAMES - LA BALLATA DELL'USIGNOLO E DEL ...,'2023-11-15'15/11/2023,USA,MEDUSA FILM S.P.A.,238466.31,32469,584902.45,79311,2023-11-17 15:31:50.351708
2,3,THE MARVELS,'2023-11-08'08/11/2023,USA,WALT DISNEY S.M.P. ITALIA,60914.35,8628,2260142.44,305291,2023-11-17 15:31:50.351708
3,4,THANKSGIVING,'2023-11-16'16/11/2023,USA,EAGLE PICTURES S.P.A.,41232.57,5709,41232.57,5709,2023-11-17 15:31:50.351708
4,5,THE OLD OAK,'2023-11-16'16/11/2023,FRA,LUCKY RED DISTRIB.,36605.89,5879,56540.79,8609,2023-11-17 15:31:50.351708


---

In [3]:
df1 = pd.DataFrame(df1)

## Cleaning the dataframe

To clean the dataframe I decided to drop duplicate values and columns that aren't relevant to my analysis, such as `df1['Incasso al 16/11/2023']` and `df1['Presenze al 16/11/2023]`. I then decided to rename the existing columns using the snake_case rule in order to make the analysis easier.

In [4]:
df1 = df1.drop_duplicates()

In [5]:
# df1 = df1.drop(columns= ['Incasso al 16/11/2023', 'Presenze al 16/11/2023'])

# at first i thought these columns weren't relevant
# making visualizations, i realized i need the data to have accurate insights on movies that started their screenings before i started to scrape the data

In [6]:
df1 = df1.rename(columns = {'Pos.':'daily_rank', 'Titolo' : 'title', 'Prima Progr.' : 'first_screening_date', 'Nazione' : 'nation', 'Distribuzione' : 'distribution', 'Incasso' : 'daily_takings', 'Presenze' : 'daily_attendance','Incasso al 16/11/2023' : 'total_takings',  'Presenze al 16/11/2023' : 'total_attendance', '2023-11-17 15:31:50.351708' : 'date_pulled'})

In [7]:
# dropping the extra 'header row' that is imported everyday when the .csv file gets updated
df1 = df1[df1['daily_rank'].str.contains('Pos.')==False]

In [8]:
# making the 'title' and 'distribution' entries consistent by appling the title format
df1['title'] = df1['title'].apply(lambda x: x.title())
df1['distribution'] = df1['distribution'].apply(lambda x: x.title())

In [9]:
# cleaning the 'first_screening_date' entries
df1['first_screening_date'] = df1['first_screening_date'].str[12:]

In [10]:
# cleaning the column showing the date the data was pulled from the site
df1['date_pulled'] = pd.to_datetime(df1['date_pulled'])
df1['date_pulled'] = pd.to_datetime(df1['date_pulled']).dt.date

#extra tries:
#df1['date_pulled'] = df1['date_pulled'].str[:10]
#df1['date_pulled'] = df1['date_pulled'].str.replace('[^a-zA-Z0-9]', '', regex = True)
#df1['date_pulled'] = df1['date_pulled'].apply(lambda x: str(x))
#df1['date_pulled'] = df1['date_pulled'].apply(lambda x: x[0:2] + '/' + x[2:4] + '/' + x[4:8])

In [11]:
from datetime import timedelta, datetime

In [12]:
df1['first_screening_date']

0      26/10/2023
1      15/11/2023
2      08/11/2023
3      16/11/2023
4      16/11/2023
          ...    
764    11/01/2024
765    26/10/2023
766    14/12/2023
767    11/01/2024
768    31/12/2023
Name: first_screening_date, Length: 700, dtype: object

In [13]:
# formatting
df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date'], errors='coerce')
df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date']).dt.date

C:\Users\Franci\AppData\Local\Temp\ipykernel_25056\2741308956.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date'], errors='coerce')


In [14]:
df1['daily_takings'] = df1['daily_takings'].astype(float)
df1['daily_attendance'] = df1['daily_attendance'].astype(int)
df1['daily_rank'] = df1['daily_rank'].astype(int)
df1['total_takings'] = df1['total_takings'].astype(float)
df1['total_attendance'] = df1['total_attendance'].astype(int)

---

## Data manipulation

I decided to create new columns to get information I may need later on during the analysis.

In [15]:
# calculating the number of days of screenings for each movie 
df1['screening_days'] = datetime.today().date() - df1['first_screening_date']
df1['screening_days'] = df1['screening_days'].map(lambda x: str(x)[:-14])

In [16]:
df1['screening_days']

0      89
1      69
2      76
3      68
4      68
       ..
764    12
765    89
766    40
767    12
768    23
Name: screening_days, Length: 700, dtype: object

In [17]:
#df1['screening_days'] = df1['screening_days'].str.strip()
df1['screening_days'] = df1['screening_days'].replace('', '0')
df1['screening_days'] = df1['screening_days'].astype(int) + 1

In [18]:
# creating a column to show the date the 'daily_takings' and 'daily_attendace' refer to
df1['date'] = df1['date_pulled'] - timedelta(days=1)
df1['date'] = pd.to_datetime(df1['date'])

In [19]:
# creating a column showing the day of the week. it'll be useful for further analysis
df1['day_of_week'] = df1['date'].dt.day_name()

In [20]:
df1['avg_ticket_price'] = df1['daily_takings'] / df1['daily_attendance']

In [21]:
pd.set_option('display.max_rows', 8)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
df1

,daily_rank,title,first_screening_date,nation,distribution,daily_takings,daily_attendance,total_takings,total_attendance,date_pulled,screening_days,date,day_of_week,avg_ticket_price
0,1,C'E' Ancora Domani,2023-10-26,ITA,Vision Distribution,449991.06,66700,15231636.85,2202169,2023-11-17,90,2023-11-16,Thursday,6.75
1,2,Hunger Games - La Ballata Dell'Usignolo E Del ...,2023-11-15,USA,Medusa Film S.P.A.,238466.31,32469,584902.45,79311,2023-11-17,70,2023-11-16,Thursday,7.34
2,3,The Marvels,2023-11-08,USA,Walt Disney S.M.P. Italia,60914.35,8628,2260142.44,305291,2023-11-17,77,2023-11-16,Thursday,7.06
3,4,Thanksgiving,2023-11-16,USA,Eagle Pictures S.P.A.,41232.57,5709,41232.57,5709,2023-11-17,69,2023-11-16,Thursday,7.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,7,C'E' Ancora Domani,2023-10-26,ITA,Vision Distribution,26249.37,4712,35628572.63,5218232,2024-01-23,90,2024-01-22,Monday,5.57
766,8,Wonka,2023-12-14,GBR,Warner Bros Italia S.P.A.,16434.08,2368,14299999.26,1906317,2024-01-23,41,2024-01-22,Monday,6.94
767,9,Enea,2024-01-11,ITA,Vision Distribution,15568.14,2438,998314.51,143483,2024-01-23,13,2024-01-22,Monday,6.39
768,10,Succede Anche Nelle Migliori Famiglie,2023-12-31,ITA,01 Distribution,11884.49,1813,5506486.74,770982,2024-01-23,24,2024-01-22,Monday,6.56


---

## Export as .csv

In [22]:
import os

In [23]:
clean_file_name = 'cinetel_clean.csv'
clean_file_path = os.path.join(os.getcwd(), clean_file_name)

In [24]:
df1.to_csv(clean_file_path, index=False)
print(f"Il DataFrame è stato esportato con successo in {clean_file_path}.")

Il DataFrame è stato esportato con successo in e:\data_analysis_python\cinetel\cinetel_clean.csv.


---

## Data Exploration

In [25]:
import seaborn as sns
import matplotlib.pyplot as plt

In [26]:
pd.set_option('display.max_rows', 8)

In [27]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 700 entries, 0 to 768
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   daily_rank            700 non-null    int32         
 1   title                 700 non-null    object        
 2   first_screening_date  698 non-null    object        
 3   nation                700 non-null    object        
 4   distribution          700 non-null    object        
 5   daily_takings         700 non-null    float64       
 6   daily_attendance      700 non-null    int32         
 7   total_takings         700 non-null    float64       
 8   total_attendance      700 non-null    int32         
 9   date_pulled           700 non-null    object        
 10  screening_days        700 non-null    int32         
 11  date                  700 non-null    datetime64[ns]
 12  day_of_week           700 non-null    object        
 13  avg_ticket_price      700

In [28]:
df1.describe()

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,date,avg_ticket_price
count,700.00,700.00,700.00,700.00,700.00,700.00,700,700.00
mean,5.50,176391.54,24466.35,5176423.87,737056.25,50.68,2023-12-19 16:27:25.714285824,7.02
min,1.00,6272.70,864.00,11652.78,1470.00,1.00,2023-11-16 00:00:00,3.70
25%,3.00,36584.03,5393.75,502089.66,72271.00,34.00,2023-12-03 00:00:00,6.48
50%,5.50,89831.49,13144.00,1835061.70,264727.00,48.00,2023-12-20 12:00:00,7.01
75%,8.00,224520.60,31601.00,4981856.12,674287.50,69.00,2024-01-05 00:00:00,7.41
max,10.00,1544231.00,211764.00,35628572.63,5218232.00,139.00,2024-01-22 00:00:00,21.18
std,2.87,222634.54,29951.27,8607879.27,1257611.91,24.95,NaN,1.19


In [29]:
df1.isnull().sum()

daily_rank              0
title                   0
first_screening_date    2
nation                  0
                       ..
screening_days          0
date                    0
day_of_week             0
avg_ticket_price        0
Length: 14, dtype: int64

In [30]:
df1.nunique()

daily_rank               10
title                    57
first_screening_date     33
nation                    8
                       ... 
screening_days           33
date                     68
day_of_week               7
avg_ticket_price        680
Length: 14, dtype: int64

In [31]:
df1.corr(numeric_only=True)

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,avg_ticket_price
daily_rank,1.00,-0.57,-0.59,-0.21,-0.20,0.04,-0.02
daily_takings,-0.57,1.00,1.00,0.18,0.17,-0.06,0.17
daily_attendance,-0.59,1.00,1.00,0.20,0.20,-0.04,0.13
total_takings,-0.21,0.18,0.20,1.00,1.00,0.49,-0.11
total_attendance,-0.20,0.17,0.20,1.00,1.00,0.50,-0.12
screening_days,0.04,-0.06,-0.04,0.49,0.50,1.00,-0.17
avg_ticket_price,-0.02,0.17,0.13,-0.11,-0.12,-0.17,1.00


In [32]:
df1.groupby('title').mean(numeric_only=True).sort_values(by = 'total_attendance', ascending= False)

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,avg_ticket_price
title,,,,,,,
C'E' Ancora Domani,3.68,309703.03,45628.75,29490297.76,4318161.13,90.00,6.60
Wonka,3.33,398492.27,52729.69,9406158.69,1252739.93,41.00,7.34
Io Capitano,8.43,15747.99,3713.14,4279651.65,747385.43,139.00,4.25
Wish,3.70,407560.87,57624.04,5156528.79,734737.56,34.00,6.92
...,...,...,...,...,...,...,...
Doc. Nelle Tue Mani - Stagione 3,7.00,21826.97,2384.00,35222.35,3855.50,36.00,9.16
Royal Opera House 2023/2024,7.00,19784.50,1916.00,35873.50,3496.00,126.00,10.33
Seventeen Tour Follow To Japan: Live Viewing,9.00,66089.87,3121.00,66089.87,3121.00,39.00,21.18
Casanova Operapop - Il Film,9.00,12093.47,1256.33,25578.93,2715.33,58.00,9.62


In [33]:
df2 = df1.groupby('title').mean(numeric_only=True).sort_values(by = 'total_attendance', ascending= False)
df2

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,avg_ticket_price
title,,,,,,,
C'E' Ancora Domani,3.68,309703.03,45628.75,29490297.76,4318161.13,90.00,6.60
Wonka,3.33,398492.27,52729.69,9406158.69,1252739.93,41.00,7.34
Io Capitano,8.43,15747.99,3713.14,4279651.65,747385.43,139.00,4.25
Wish,3.70,407560.87,57624.04,5156528.79,734737.56,34.00,6.92
...,...,...,...,...,...,...,...
Doc. Nelle Tue Mani - Stagione 3,7.00,21826.97,2384.00,35222.35,3855.50,36.00,9.16
Royal Opera House 2023/2024,7.00,19784.50,1916.00,35873.50,3496.00,126.00,10.33
Seventeen Tour Follow To Japan: Live Viewing,9.00,66089.87,3121.00,66089.87,3121.00,39.00,21.18
Casanova Operapop - Il Film,9.00,12093.47,1256.33,25578.93,2715.33,58.00,9.62
